# HW3:Use LSTM & CNN model to classify customized candlestick pattern (at least 3 classes)

## 1.程式碼

### (1)Use LSTM model to classify customized candlestick pattern

* candlestick_lstm_training_r09723032_江奕萱.py

In [4]:
from sklearn.metrics import confusion_matrix
import pickle
import keras
from keras.layers import LSTM
from keras.layers import Dense, Activation, Conv2D, MaxPool2D, Dropout, Flatten
from keras.datasets import mnist
from keras.models import Sequential
from keras.optimizers import Adam


#讀取資料的部分
def load_data(name):
    # load data from data folder
    with open(name, 'rb') as f:
        data = pickle.load(f)
    return data

'''
資料前處理 Dataset
'''
def lstm_preprocess(x_train, x_test, y_train, y_test, n_step, n_input, n_classes):
    x_train = x_train.reshape(-1, n_step, n_input)
    x_test = x_test.reshape(-1, n_step, n_input)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255 #標準化數據
    x_test /= 255 #標準化數據
    y_train = keras.utils.to_categorical(y_train, n_classes)
    y_test = keras.utils.to_categorical(y_test, n_classes)
    return (x_train, x_test, y_train, y_test)


'''
建立LSTM模型:
主要是用一層LSTM以及一層的dense進行預測，
output使用softmax作為計算的fuction

模型調整:
新增Dense(128, activation='relu')
新增Dense(64, activation='relu')
'''
def lstm_model(n_input, n_step, n_hidden, n_classes):
    #添加LSTM、Dense層
    model = Sequential()
    model.add(LSTM(n_hidden, batch_input_shape=(None, n_step, n_input), unroll=True))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(n_classes, activation='softmax'))
    
    return model

'''
訓練模型:
先設定優化器Adam
再來設定模型的Loss函數、優化器以及用來判斷模型好壞的依據（metrics），此處使用accuracy
最後訓練模型
'''
def lstm_train(model, x_train, y_train, x_test, y_test,learning_rate, training_iters, batch_size):
    adam = Adam(lr=learning_rate)
    model.summary()
    model.compile(optimizer=adam,loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(x_train, y_train,batch_size=batch_size, epochs=training_iters,verbose=1, validation_data=(x_test, y_test))
'''
辨別機器學習模型的好壞，印出Confusion Matrix:
先得到模型的預測結果
然後與真正的答案做對照
由左上至右下的對角線，代表True Positive和True Negative，
亦即模型預測成功的狀況，因此數字越高越好
'''
def lstm_result(data, x_train, x_test, model):
    # get train & test pred-labels
    train_pred = model.predict_classes(x_train)
    test_pred = model.predict_classes(x_test)
    # get train & test true-labels
    train_label = data['train_label'][:, 0]
    test_label = data['test_label'][:, 0]
    # confusion matrix
    train_result_cm = confusion_matrix(train_label, train_pred, labels=range(9))
    test_result_cm = confusion_matrix(test_label, test_pred, labels=range(9))
    print(train_result_cm, '\n'*2, test_result_cm)

'''
補充說明:
Batch_Size 設越大，
則跑完一個 Epoch 的時間大約成比例縮小(因為要跑的 Iteration就比例減少)
但設的太大，就必須放大 Epoch 數。
這是因為Batch_Size大，
一個 Epoch 可以跑得 Iteration 數就成比例變少，
就沒有足夠的梯度下降讓損失函數到平穩的低點。
所以必須加大 Epoch 數，這樣訓練時間又變長了，
取捨之間也是必須用觀察的。

參數調整:
learning_rate : 0.001->0.01
batch_size: 128->32
'''
def candlestick_lstm():
    # training parameters
    learning_rate = 0.01 #學習率
    training_iters = 10
    batch_size = 32 # BATCH的大小，相當於一次處理的個數

    # model parameters
    n_input = 40 
    n_step = 10
    n_hidden = 256 #隱含層的特徵數
    n_classes = 10 

    data = load_data('./data/label8_eurusd_10bar_1500_500_val200_gaf_culr.pkl')
    x_train, y_train, x_test, y_test = data['train_gaf'], data['train_label'][:, 0], data['test_gaf'], data['test_label'][:, 0]
    x_train, x_test, y_train, y_test = lstm_preprocess(x_train, x_test, y_train, y_test, n_step, n_input, n_classes)

    model = lstm_model(n_input, n_step, n_hidden, n_classes)
    lstm_train(model, x_train, y_train, x_test, y_test, learning_rate,training_iters, batch_size)
    scores = model.evaluate(x_test, y_test, verbose=0)
    print('LSTM test accuracy:', scores[1])
    lstm_result(data, x_train, x_test, model)


### (2)Use CNN model to classify customized candlestick pattern

* candlestick_cnn_training_r09723032_江奕萱.py

In [15]:
from sklearn.metrics import confusion_matrix
import numpy as np
import pickle
from keras import backend as K
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, Activation, MaxPool2D


#讀取資料
def load_data(name):
    # load data from data folder
    with open(name, 'rb') as f:
        data = pickle.load(f)
    return data

'''
建立CNN模型：
先建立了 Convolution 層(兩層)，
接著用 Flattern 攤平維度，
然後接 Dense 全連接層三層，
最後輸出9個類別
'''
# Model Structure
def cnn_model(params):
    # initializing CNN
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu', input_shape=(10, 10, 4)))
    # Second convolutional layer
    model.add(Conv2D(filters=48, kernel_size=(5,5), padding='valid', activation='relu')) 
    #將 feature maps 攤平放入一個向量中
    model.add(Flatten()) #攤平維度
    model.add(Dense(256, activation='relu'))
    model.add(Dense(84, activation='relu'))
    model.add(Dense(9, activation='softmax'))
    return model


    
'''
訓練模型：
選擇使用CNN Model
設定模型的Loss函數、優化器以及用來判斷模型好壞的依據（metrics），這裡用準確度來衡量
最後訓練模型
'''
def cnn_train(params, data):
    model = cnn_model(params)
    # 設定模型的Loss函數、優化器以及用來判斷模型好壞的依據（metrics）
    model.compile(loss='categorical_crossentropy', optimizer=params['optimizer'], metrics=['accuracy'])
    #訓練模型
    hist = model.fit(x=data['train_gaf'], y=data['train_label_arr'],batch_size=params['batch_size'], epochs=params['epochs'], verbose=2)
    return (model, hist)

'''
辨別機器學習模型的好壞，印出Confusion Matrix：
先得到模型的預測結果
然後與真正的答案做對照
由左上至右下的對角線，代表True Positive和True Negative，
亦即模型預測成功的狀況，因此數字越高越好
'''

def cnn_result(data, model):
    # get train & test pred-labels
    train_pred = model.predict_classes(data['train_gaf'])
    test_pred = model.predict_classes(data['test_gaf'])
    # get train & test true-labels
    train_label = data['train_label'][:, 0]
    test_label = data['test_label'][:, 0]
    # confusion matrix
    train_result_cm = confusion_matrix(train_label, train_pred, labels=range(9))
    test_result_cm = confusion_matrix(test_label, test_pred, labels=range(9))
    print(train_result_cm, '\n'*2, test_result_cm)

'''
補充說明:
Batch_Size 設越大，
則跑完一個 Epoch 的時間大約成比例縮小(因為要跑的 Iteration就比例減少)
但設的太大，就必須放大 Epoch 數。
這是因為Batch_Size大，
一個 Epoch 可以跑得 Iteration 數就成比例變少，就
沒有足夠的梯度下降讓損失函數到平穩的低點。
所以必須加大 Epoch 數，這樣訓練時間又變長了，
取捨之間也是必須用觀察的。

參數調整:
learning_rate : 0.01->0.05
batch_size: 64->32
'''
def candlestick_cnn():
    PARAMS = {}
    PARAMS['pkl_name'] = './data/label8_eurusd_10bar_1500_500_val200_gaf_culr.pkl' 
    # training parameters 
    PARAMS['classes'] = 9 
    PARAMS['lr'] = 0.05  #學習率
    PARAMS['epochs'] = 10  #決定訓練要跑幾回合 epoch，一個 epoch 就是全部的訓練數據都被掃過一遍。
    PARAMS['batch_size'] = 32 # BATCH的大小，相當於一次處理的個數
    PARAMS['optimizer'] = optimizers.SGD(lr=PARAMS['lr'])
    # load data & keras model
    data = load_data(PARAMS['pkl_name'])
    # train cnn model
    model, hist = cnn_train(PARAMS, data)
    # 驗證模型
    scores = model.evaluate(data['test_gaf'], data['test_label_arr'], verbose=0)
    print('CNN test accuracy:', scores[1])
    cnn_result(data, model)
    


## 2.執行

### (1)LSTM ，印出預測準確度以及Confusion Matrix

In [6]:
candlestick_lstm()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 256)               304128    
_________________________________________________________________
dense (Dense)                (None, 128)               32896     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                650       
Total params: 345,930
Trainable params: 345,930
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
469/469 [==============================] - 22s 33ms/step - loss: 1.7555 - accuracy: 0.3209 - val_loss: 0.7226 - val_accuracy: 0.7244
Epoch 2/10
469/469 [==============================] - 13s 27ms/step - loss: 0.6588 - accuracy: 0.7425 - 

C:\Users\ben82\anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[[2312  114   73  122   71   86   55   97   70]
 [  12 1483    0    5    0    0    0    0    0]
 [ 110    0 1382    0    8    0    0    0    0]
 [  25    8    0 1363    0    0    0  104    0]
 [ 142    0    9    0 1318    0    0    0   31]
 [  83    0    0    0    0 1400    0   17    0]
 [ 154    1    1    0    3    0 1325    0   16]
 [  22    0    0  140    0    8    0 1330    0]
 [ 101    0    2    0  177    0   29    0 1191]] 

 [[762  42  27  35  23  32  15  39  25]
 [  9 489   0   2   0   0   0   0   0]
 [ 30   0 469   0   1   0   0   0   0]
 [  9  10   0 442   0   0   0  39   0]
 [ 46   0  10   0 433   0   0   0  11]
 [ 33   0   0   0   0 464   0   3   0]
 [ 65   0   1   0   0   0 431   0   3]
 [  7   1   0  32   0   2   0 458   0]
 [ 21   0   0   0  51   0   7   0 421]]


* 調整模型和參數之後，準確度提高:80%->87%

### (2)CNN ，印出預測準確度以及Confusion Matrix

In [16]:
candlestick_cnn()

Epoch 1/10
469/469 - 7s - loss: 0.8790 - accuracy: 0.6592
Epoch 2/10
469/469 - 6s - loss: 0.5205 - accuracy: 0.8042
Epoch 3/10
469/469 - 6s - loss: 0.4529 - accuracy: 0.8301
Epoch 4/10
469/469 - 6s - loss: 0.4182 - accuracy: 0.8479
Epoch 5/10
469/469 - 6s - loss: 0.3872 - accuracy: 0.8567
Epoch 6/10
469/469 - 6s - loss: 0.3585 - accuracy: 0.8676
Epoch 7/10
469/469 - 6s - loss: 0.3316 - accuracy: 0.8783
Epoch 8/10
469/469 - 6s - loss: 0.3122 - accuracy: 0.8875
Epoch 9/10
469/469 - 6s - loss: 0.2958 - accuracy: 0.8920
Epoch 10/10
469/469 - 6s - loss: 0.2714 - accuracy: 0.9013
CNN test accuracy: 0.8809999823570251
[[2476   75   58   76   58   42  100   80   35]
 [   4 1496    0    0    0    0    0    0    0]
 [  12    0 1488    0    0    0    0    0    0]
 [   7   12    0 1419    0    0    0   62    0]
 [  30    0   31    0 1379    0    1    0   59]
 [  49    0    0    1    0 1428    0   22    0]
 [   7    0    0    0    0    0 1490    0    3]
 [   4    1    0   92    0    1    0 1402    

* 調整參數之後，準確度提高:80%->88%